In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install fake_useragent

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import numpy as np

import re

#webscraping
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [12]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [13]:
def get_state_df(state):
    '''
    A fucntion for scraping TripAdvisor to get all info for top 30 attractions for one state

    Parameters
    ----------
    state as a string

    Returns
    -------
    df containing all top attractions and images for a state
    '''
    URL = "https://www.tripadvisor.com/Attractions/"
    #driver = webdriver.Chrome(executable_path=chromedriver)
    options = webdriver.ChromeOptions()
    options.add_argument("--log-path=/path/to/chromedriver.log")
    driver = webdriver.Chrome(options=options)



    driver.get(URL)
    time.sleep(2)  #pause to be sure page has loaded

    #find state with search on main page
    # search = driver.find_element_by_name('q')
    # search.send_keys(state + ", United States"  )
    search = driver.find_element(By.NAME, 'q')
    search.send_keys(state + ", India")

    time.sleep(2)
    search.send_keys(Keys.DOWN)
    if state == 'Delhi':
        search.send_keys(Keys.DOWN)
    search.send_keys(Keys.RETURN)

    # click Things to Do button
    #things_to_do_button = driver.find_elements_by_xpath('//*[contains(text() , "Things to Do")]')[0]
    # things_to_do_button = driver.find_elements(By.XPATH, '//*[contains(text() , "Things to Do")]')[0]
#     things_to_do_button = WebDriverWait(driver, 50).until(
#     EC.element_to_be_clickable((By.XPATH, '//*[contains(text() , "Things to Do")]'))
# )
    print("Waiting for Things to Do button...")
    things_to_do_button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//*[contains(text() , "Top Attractions in")]'))
)
    print("Things to Do button is clickable.")

    things_to_do_button.click()
    time.sleep(2)

    error = 0
    try:
        # click second See All button
        print("Waiting for See all")
        see_all_button = driver.find_elements(By.XPATH,'//*[contains(text() , "See all")]')[1]
        print("See all is clickable")
        see_all_button.click()
        time.sleep(2)
    except:
        error = error + 1

    #check if second window appears, if yes switch back to main window and click see more button
    try:
        print('asd')
        driver.switch_to.window(driver.window_handles[1])

        #driver.switch_to.window(driver.window_handles[0])
        print('ftyguhj')
        #click See more button
        #attraction_button = driver.find_elements_by_xpath('//*[contains(text() , "See more")]')[0]
        attraction_button = driver.find_elements(By.XPATH, '//*[contains(text() , "See more")]')[0]

        attraction_button.click()

        #initalize BeautifulSoup
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "lxml")

    except:
        #initalize BeautifulSoup
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "lxml")

    url = "https://www.tripadvisor.com"
    links = {}
    body = soup.find_all('body')[0]
    link_list = body.find_all( 'a') #find all links on page

    #loop through range to get numbers 1-30, we want top 30 attractions of each state, then for each number
    #loop through links, add attraction name and link to links dictionary if number match outside loop
    for x in range(1,11):
        find = str(x) + "[.] "
        for l in link_list:
            if re.search(find,l.text):
                name = l.text
                name = re.sub(r'^.*? ', '', name) #clean name
                links[name] = url + l['href']

    driver.quit() #close browser

    return get_state_attraction_info(links, state)

In [21]:
def get_state_attraction_info(link_dict, state):
    '''
    A fucntion for scraping TripAdvisor to get locations and image links for top 30 attractions for one state

    Parameters
    ----------
    dictionary, key is attraction name and value is link

    Returns
    -------
    df containing all top attractions, locations, and images for a state
    '''
    df = pd.DataFrame(columns = ['name', 'location', 'state', 'img_num'])
    idx = 0
    img_link_dict = {}
    print(link_dict)
    ua = UserAgent()
    headers = {'user-agent': ua.random}

    #loop through dictionary append info to df
    for key, value in link_dict.items():

        response = requests.get(value, headers = headers)
        print(value)
        page= response.text
        soup = BeautifulSoup(page, "lxml")
        time.sleep(1)  #pause to be sure page has loaded

        #try to find attraction location, return state name if none
        try:
            body = soup.find('div', {'data-tab' :'TABS_LOCATION'})
            divs = body.find_all('span')
            address = divs[3].text
        except:
            address = state
        
        print('wadsfg')
        img_link_list = get_photo_links(value)
        img_link_dict[key] = img_link_list

        # df = df.append(pd.DataFrame({'name':key, 'location': address,'state': state , 'img_num' : len(img_link_list)},
        #                             index=[idx]), ignore_index=True)
        df = pd.concat([df, pd.DataFrame({'name': key, 'location': address, 'state': state, 'img_num': len(img_link_list)}, index=[idx])], ignore_index=True)
        print(df)
        idx = idx + 1

        #change user agent, chosen randomly every 3 loops
        if idx%3 == 0:
            headers = {'user-agent': ua.random}

    return (df,img_link_dict)

In [27]:
def get_photo_links(URL):
    '''
    A fucntion for scraping TripAdvisor to get all image links in a gallery for a specified attraction

    Parameters
    ----------
    url of attraction

    Returns
    -------
    list containing all image links for an attraction
    '''
    #driver = webdriver.Chrome(chromedriver)
    options = webdriver.ChromeOptions()
    options.add_argument("--log-path=/path/to/chromedriver.log")
    driver = webdriver.Chrome(options=options)
    driver.get(URL)
    time.sleep(2)  #pause to be sure page has loaded

    try:
        #click All photos button
        all_photos_button = driver.find_elements(By.XPATH,'//*[contains(text() , "All photos")]')[0]
        all_photos_button.click()
        time.sleep(2)
    except:
        return []

    error = 0
    try:
        #click first photo
        #all_photos_button = driver.find_elements_by_class_name("photoGridImg")[0]
        all_photos_button = driver.find_elements(By.CLASS_NAME,"photoGridImg")[0]
        all_photos_button.click()
        time.sleep(1)
    except:
        error = error +1

    #initalize BeautifulSoup
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")
    time.sleep(1)

    images = soup.find_all('img') #find all imgs
    #loop through imgs, check for 2 image types, add to list if it is a TripAdvisor user uploaded photo
    img_links = []
    errors = 0
    for i in images[:-1]:
        try:
            try:
                link = i.attrs['data-lazyurl']
                if re.search('photo',link):
                    img_links.append(link)
            except:
                link = i.attrs['src']
                if re.search('photo',link):
                    img_links.append(link)
        except:
            errors = errors + 1

    driver.quit()
    
    
    return img_links


In [28]:
loc_df = pd.DataFrame(columns = ['name', 'location', 'img_num'])
img_dict = {}

In [37]:
states = ["Bengaluru","Mysore","Chennai","Kochi","Mumbai","Pune","Vishakapatnam","Surat","Kashmir","Manali"]
# ["Alabama"]
          #"Arkansas", "Arizona","California","Colorado","Connecticut", "Washington DC", "Delaware"]
          # "Florida", "Georgia", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas","Kentucky", "Louisiana",
          # "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana",
          # "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada",
          # "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
          # "Tennessee","Texas","Utah","Virginia","Vermont","Washington","Wisconsin","West Virginia", "Wyoming"]


for s in states:
    df, images = get_state_df(s)
    loc_df = pd.concat([loc_df, df], axis=0, ignore_index=True)
    img_dict.update(images)

Waiting for Things to Do button...
Things to Do button is clickable.
Waiting for See all
See all is clickable
asd
{'Wonderla Bengaluru': 'https://www.tripadvisor.com/Attraction_Review-g297628-d1603947-Reviews-Wonderla_Bengaluru-Bengaluru_Bangalore_District_Karnataka.html', 'Ragigudda Sri Prasanna Anjaneyaswamy Temple': 'https://www.tripadvisor.com/Attraction_Review-g297628-d4401144-Reviews-Ragigudda_Sri_Prasanna_Anjaneyaswamy_Temple-Bengaluru_Bangalore_District_Karnatak.html', 'ISKCON Temple Bangalore': 'https://www.tripadvisor.com/Attraction_Review-g297628-d325162-Reviews-ISKCON_Temple_Bangalore-Bengaluru_Bangalore_District_Karnataka.html', 'Vidhana Soudha and Attara Kacheri': 'https://www.tripadvisor.com/Attraction_Review-g297628-d2587646-Reviews-Vidhana_Soudha_and_Attara_Kacheri-Bengaluru_Bangalore_District_Karnataka.html', 'Lalbagh Botanical Garden': 'https://www.tripadvisor.com/Attraction_Review-g297628-d631367-Reviews-Lalbagh_Botanical_Garden-Bengaluru_Bangalore_District_Karnatak

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="q"]"}
  (Session info: chrome=120.0.6099.217); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF640002142+3514994]
	(No symbol) [0x00007FF63FC20CE2]
	(No symbol) [0x00007FF63FAC76AA]
	(No symbol) [0x00007FF63FB11860]
	(No symbol) [0x00007FF63FB1197C]
	(No symbol) [0x00007FF63FB54EE7]
	(No symbol) [0x00007FF63FB3602F]
	(No symbol) [0x00007FF63FB528F6]
	(No symbol) [0x00007FF63FB35D93]
	(No symbol) [0x00007FF63FB04BDC]
	(No symbol) [0x00007FF63FB05C64]
	GetHandleVerifier [0x00007FF64002E16B+3695259]
	GetHandleVerifier [0x00007FF640086737+4057191]
	GetHandleVerifier [0x00007FF64007E4E3+4023827]
	GetHandleVerifier [0x00007FF63FD504F9+689705]
	(No symbol) [0x00007FF63FC2C048]
	(No symbol) [0x00007FF63FC28044]
	(No symbol) [0x00007FF63FC28

In [38]:
loc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      64 non-null     object
 1   location  64 non-null     object
 2   img_num   64 non-null     object
 3   state     64 non-null     object
dtypes: object(4)
memory usage: 2.1+ KB


In [39]:
loc_df.to_pickle('attractions_loc_df.pkl')

In [40]:
img_df = pd.DataFrame.from_dict(img_dict,orient='index')
img_df.to_pickle('attractions_img_links_df.pkl')

In [41]:
np.sum(img_df.count()) #number of img links

2791

In [43]:
import pickle
with open('attractions_img_links_df.pkl','rb')as f:
    data = pickle.load(f)

In [44]:
data

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
Wonderla Bengaluru,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Ragigudda Sri Prasanna Anjaneyaswamy Temple,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,...,None,None,None,None,None,None,None,None,None,None
ISKCON Temple Bangalore,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,None,None,None,None,None,None,None,None
Vidhana Soudha and Attara Kacheri,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Lalbagh Botanical Garden,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
UB City,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,...,None,None,None,None,None,None,None,None,None,None
Cubbon Park,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Bannerghatta National Park,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,https://dynamic-media-cdn.tripadvisor.com/medi...,...,None,None,None,None,None,None,None,None,None,None
Mahatma Gandhi (MG) Road,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Visvesvaraya Industrial and Technological Museum,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [45]:
import pickle
with open('attractions_loc_df.pkl','rb')as f:
    data1 = pickle.load(f)

In [46]:
data1

,name,location,img_num,state
0,Wonderla Bengaluru,Bengaluru,0,Bengaluru
1,Ragigudda Sri Prasanna Anjaneyaswamy Temple,Bengaluru,100,Bengaluru
2,ISKCON Temple Bangalore,Bengaluru,0,Bengaluru
3,Vidhana Soudha and Attara Kacheri,Bengaluru,127,Bengaluru
4,Lalbagh Botanical Garden,Bengaluru,0,Bengaluru
...,...,...,...,...
59,Arulmigu Thiruvallikeni Parthasarathy Temple,Chennai,99,Chennai
60,Express Avenue Mall,Chennai,0,Chennai
61,Shirdi Sai Baba Temple,Chennai,0,Chennai
62,Mylapore,Chennai,100,Chennai
